In [1]:
cd("/home/michael/work/github/epinets")

In [2]:
include("EpiSim.jl")

Main.EpiSim

In [3]:
using CSV
using Plots
using LightGraphs
using JLD2, FileIO
using Statistics

In [4]:
cd("/Users/michael/work/github/COVID-19/csse_covid_19_data/csse_covid_19_time_series")

LoadError: IOError: chdir /Users/michael/work/github/COVID-19/csse_covid_19_data/csse_covid_19_time_series: no such file or directory (ENOENT)

In [5]:
file="time_series_covid19_confirmed_global.csv"
allstates=["Australian Capital Territory" "New South Wales"  "Northern Territory" "Queensland" "South Australia" "Tasmania" "Victoria" "Western Australia"]

1×8 Array{String,2}:
 "Australian Capital Territory"  …  "Victoria"  "Western Australia"

In [6]:
z=[]
for state in allstates
    z=push!(z,EpiSim.getdata("Australia",state))
end

LoadError: ArgumentError: "time_series_covid19_confirmed_global.csv" is not a valid file

In [7]:
ddays=CSV.File(file)[1]
ddays=propertynames(ddays,4)[5:end]
ddays=String.(ddays)
ndays=length(ddays)

LoadError: ArgumentError: "time_series_covid19_confirmed_global.csv" is not a valid file

In [8]:
pops=[426709 8089526 245869 5095100 1751693 534281 6594804 2621680]

1×8 Array{Int64,2}:
 426709  8089526  245869  5095100  1751693  534281  6594804  2621680

In [9]:
#'reasonable' parameters
epiparam=Dict()
epiparam["p0"]=0.2 #a guess - tuned to match observed data 
epiparam["p2"]=1/12 #revised infection rate with distancing measure
epiparam["q"]=1/7 #"up to" two weeks
epiparam["r0"]=1/14 #about two weeks for mild, 3-6 for severe
epiparam["r2"]=1/4 #revised removal rate (now due to testing and isolation)
epiparam["nseeds"]=5 #probably too many, consider dropping.

5

In [10]:
i=8
epiparam["pop"]=Int(floor(sqrt(pops[i])))^2
epiparam["gridsize"]=Int(floor(sqrt(pops[i])))
y=z[i];

LoadError: BoundsError: attempt to access 0-element Array{Any,1} at index [8]

In [11]:
ddt(z,zt)=count(z->z>0, z[1:zt])+ count(z->z<0, z[zt+1:end])
~,tpday=findmax([ddt(diff(diff(y)),nx) for nx in 1:(ndays-2)])
#this is the turning point between exponential growth and decay. totItp total infections at day tpday
totItp=y[tpday+1]
plot(1:tpday+1,y[1:tpday+1],lw=4,label="growth phase",title=allstates[i])
plot!(tpday+1:ndays,y[tpday+1:ndays],lw=4,label="plateau")

LoadError: UndefVarError: ndays not defined

In [12]:
gridsize=epiparam["gridsize"]
bamodel=barabasi_albert(gridsize^2, 3, 2)
lattice=LightGraphs.grid((gridsize,gridsize),periodic=true)
wattstrog95=watts_strogatz(gridsize^2, 4, 0.013)  #s=0.013 => 95% compliance
wattstrog90=watts_strogatz(gridsize^2, 4, 0.026)  #s=0.026 => 90% compliance
wattstrog80=watts_strogatz(gridsize^2, 4, 0.053)  #s=0.053 => 80% compliance

{2621161, 5242322} undirected simple Int64 graph

In [13]:
St90,Et90,It90,Rt90=EpiSim.episim(bamodel,wattstrog90, epiparam, totItp, tpday+2, 180, 100)

LoadError: UndefVarError: tpday not defined

In [14]:
St95,Et95,It95,Rt95=EpiSim.episim(bamodel,wattstrog95, epiparam, totItp, tpday+2, 180, 100)

LoadError: UndefVarError: tpday not defined

In [15]:
St80,Et80,It80,Rt80=EpiSim.episim(bamodel,wattstrog80, epiparam, totItp, tpday+2, 180, 100)

LoadError: UndefVarError: tpday not defined

In [16]:
using Dates 

In [17]:
df=DateFormat("d u")

dateformat"d u"

In [18]:
datadates=Date.(ddays,"m/d/y")

LoadError: UndefVarError: ddays not defined

In [19]:
plotly()
plot(datadates,y)

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/michael/.julia/packages/Plots/6EMd6/src/backends.jl:372


LoadError: UndefVarError: datadates not defined

In [20]:
plot(y)

LoadError: UndefVarError: y not defined

In [21]:
function epipred(th,St,Et,It,Rt)
    #find the timestep of the threshold
    (lenp,nsims)=size(St)
    tstep=Array{Int,1}(undef,nsims)
    nbad=0
    for i in 1:nsims
        j=1
        while It[j,i]+Rt[j,i]<th
            j += 1
            if j>lenp
                j=0
                nbad += 1
                break
            end
        end
        tstep[i]=j
    end
    ndays=lenp-maximum(tstep)
    Sp=Array{UInt64,2}(undef,ndays,nsims-nbad)
    Ep=Array{UInt64,2}(undef,ndays,nsims-nbad)
    Ip=Array{UInt64,2}(undef,ndays,nsims-nbad)
    Rp=Array{UInt64,2}(undef,ndays,nsims-nbad)
    i=1
    ii=1
    while i<nsims-nbad+1
        if tstep[ii]>0
            Sp[:,i] = St[tstep[ii]:(tstep[ii]+ndays-1) ,i]
            Ep[:,i] = Et[tstep[ii]:(tstep[ii]+ndays-1) ,i]
            Ip[:,i] = It[tstep[ii]:(tstep[ii]+ndays-1) ,i]
            Rp[:,i] = Rt[tstep[ii]:(tstep[ii]+ndays-1) ,i]
            i += 1
            ii += 1
        else
            ii += 1
        end
    end
    return (Sp,Ep,Ip,Rp) 
end


epipred (generic function with 1 method)

In [22]:
(Sp,Ep,Ip,Rp) = epipred(400, St80,Et80,It80,Rt80)

LoadError: UndefVarError: St80 not defined

In [23]:
    function plotqnt(x,y,col,labl,qnt::Float64=0.45)
        nt,ny=size(y)
        low=Array{Float64,1}(undef,nt)
        mid=Array{Float64,1}(undef,nt)
        hig=Array{Float64,1}(undef,nt)
        for i in 1:nt
            low[i],mid[i],hig[i] = quantile(y[i,:],[0.5-qnt, 0.5, 0.5+qnt])
        end
        plot!(x,mid,grid=false,ribbon=(mid-low,hig-mid),fillalpha=.25,lw=3, seriescolor=col, label=labl)
    end

plotqnt (generic function with 2 methods)

In [24]:
lnp,~=size(Ip)
plot(y)
plotqnt(73:72+lnp,Ip+Rp,:blue,"90% Compliance")

LoadError: UndefVarError: Ip not defined

In [25]:
th=355
tp=count(x->x<th,y)

LoadError: UndefVarError: y not defined

In [26]:
ddt(z,zt)=count(z->z>0, z[1:zt])+ count(z->z<0, z[zt+1:end])
~,tpday=findmax([ddt(diff(diff(y)),nx) for nx in 1:(ndays-2)])
#this is the turning point between exponential growth and decay. totItp total infections at day tpday
totItp=y[tpday+1]
plot(ddays[1:tpday+1],y[1:tpday+1],lw=4,label="growth phase",title=allstates[i],size=(1200,800))
plot!(ddays[tpday+1:ndays],y[tpday+1:ndays],lw=4,label="plateau")
(Sp,Ep,Ip,Rp) = epipred(th, St80,Et80,It80,Rt80)
lnp,~=size(Ip)
plotqnt((tp+1):tp+lnp,Ip+Rp,:blue,"80% Compliance")
(Sp,Ep,Ip,Rp) = epipred(th, St90,Et90,It90,Rt90)
lnp,~=size(Ip)
plotqnt((tp+1):tp+lnp,Ip+Rp,:green,"90% Compliance")
(Sp,Ep,Ip,Rp) = epipred(th, St95,Et95,It95,Rt95)
lnp,~=size(Ip)
plotqnt((tp+1):tp+lnp,Ip+Rp,:red,"95% Compliance")

LoadError: UndefVarError: ndays not defined

In [27]:
iso=0.95
pop=epiparam["pop"]
grd=Int(floor(sqrt(Int(floor(pop*(1-iso))))))
isosize=pop-grd^2
sg=SimpleGraph(isosize)
ws=watts_strogatz(grd^2, 4, 0.013)
isograph95=union(sg,ws)

{2490117, 262088} undirected simple Int64 graph

In [28]:
Int(floor(sqrt(Int(floor(pop*(1-iso))))))

362

In [29]:
St9x,Et9x,It9x,Rt9x=EpiSim.episim3(bamodel,wattstrog95, isograph95, epiparam, totItp, tpday+2, 1, 180, 100)

LoadError: UndefVarError: tpday not defined

In [30]:
iso=0.80
pop=epiparam["pop"]
grd=Int(floor(sqrt(Int(floor(pop*(1-iso))))))
isosize=pop-grd^2
sg=SimpleGraph(isosize)
ws=watts_strogatz(grd^2, 4, 0.013)
isograph80=union(sg,ws)

{2096985, 1048352} undirected simple Int64 graph

In [31]:
iso=0.90
pop=epiparam["pop"]
grd=Int(floor(sqrt(Int(floor(pop*(1-iso))))))
isosize=pop-grd^2
sg=SimpleGraph(isosize)
ws=watts_strogatz(grd^2, 4, 0.013)
isograph90=union(sg,ws)

{2360040, 522242} undirected simple Int64 graph

In [32]:
St8,Et8,It8,Rt8=EpiSim.episim(bamodel,isograph80, epiparam, totItp, tpday+2, 180, 100)
St9,Et9,It9,Rt9=EpiSim.episim(bamodel,isograph90, epiparam, totItp, tpday+2, 180, 100)

LoadError: UndefVarError: tpday not defined

In [33]:
ddt(z,zt)=count(z->z>0, z[1:zt])+ count(z->z<0, z[zt+1:end])
~,tpday=findmax([ddt(diff(diff(y)),nx) for nx in 1:(ndays-2)])
#this is the turning point between exponential growth and decay. totItp total infections at day tpday
totItp=y[tpday+1]
plot(ddays[1:tpday+1],y[1:tpday+1],lw=4,label="growth phase",title="Perth, Western Australia",size=(1800,1200))
plot!(ddays[tpday+1:ndays],y[tpday+1:ndays],lw=4,seriescolor=:black,label="plateau")
(Sp,Ep,Ip,Rp) = epipred(th, St80,Et80,It80,Rt80)
lnp,~=size(Ip)
plotqnt((tp+1):tp+lnp,Ip+Rp,:red,"80% distancing 21/3")
(Sp,Ep,Ip,Rp) = epipred(th, St8,Et8,It8,Rt8)
lnp,~=size(Ip)
plotqnt((tp+1):tp+lnp,Ip+Rp,:green,"80% isolating 30/3")
(Sp,Ep,Ip,Rp) = epipred(th, St90,Et90,It90,Rt90)
lnp,~=size(Ip)
plotqnt((tp+1):tp+lnp,Ip+Rp,:orange,"90% distancing")
#(Sp,Ep,Ip,Rp) = epipred(th, St9,Et9,It9,Rt9)
#lnp,~=size(Ip)
#plotqnt((tp+1):tp+lnp,Ip+Rp,:blue,"90% isolating 23/3")
#(Sp,Ep,Ip,Rp) = epipred(th, St9x,Et9x,It9x,Rt9x)
#lnp,~=size(Ip)
#plotqnt((tp+1):tp+lnp,Ip+Rp,:yellow,"95% isolating 21/3")
plot!(ddays[tpday+1:ndays],y[tpday+1:ndays],lw=4,seriescolor=:black,label=false,xlimit=(0,150),ylimit=(0,3000))



LoadError: UndefVarError: ndays not defined

In [34]:
cd("/Users/michael/work/github/epinets")

LoadError: IOError: chdir /Users/michael/work/github/epinets: no such file or directory (ENOENT)